In [2]:
%load_ext autoreload
%autoreload 2


import modish
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities_motifs'

import flotilla

study = flotilla.embark('singlecell_pnm_figure2_modalities')
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))
modality_order = ['~0', 'middle', '~1', 'bimodal', 'ambiguous']
modality_colors = dict(zip(modality_order, sns.color_palette('deep')))

modality_factorplot_kws = dict(hue_order=modality_order, order=study.phenotype_order)

2015-09-30 10:50:53	Reading datapackage from /home/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities/datapackage.json
2015-09-30 10:50:53	Parsing datapackage to create a Study object
2015-09-30 10:52:03	Initializing Study
2015-09-30 10:52:03	Initializing Predictor configuration manager for Study
2015-09-30 10:52:03	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2015-09-30 10:52:03	Added ExtraTreesClassifier to default predictors
2015-09-30 10:52:03	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2015-09-30 10:52:03	Added ExtraTreesRegressor to default predictors
2015-09-30 10:52:03	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2015-09-30 10:52:03	Added GradientBoostingClassifier to default predictors
2015-09-30 10:52:03	Predictor GradientBoostingRegressor is of type <class 'sklearn.ensemble.gradient_boost

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


### Make BED files of each modality within each celltype

Use the other modalities, within that celltype, as background.

In [3]:
%%time 

bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities'
! mkdir $bed_folder
import pybedtools

DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/skipped_exon_exon2.bed'.format(DIR)
exon_bed = pybedtools.BedTool(exon_bedfile)

for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    background_events = set(phenotype_df.event_name)
    for modality, modality_df in phenotype_df.groupby('modality'):
        event_names = set(modality_df.event_name)
        format_args = bed_folder, phenotype, modality
        filtered_bed = exon_bed.filter(lambda x: x.name in event_names)
        filtered_filename = '{}/skipped_exon_exon2_{}_{}.bed'.format(*format_args)
        filtered_bed.saveas(filtered_filename)

        background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
        background_filename = '{}/skipped_exon_exon2_{}_{}_background.bed'.format(*format_args)
        background_bed.saveas(background_filename)

        ! wc -l $filtered_filename
        ! wc -l $background_filename

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities': File exists
1071 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_bimodal.bed
9028 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_bimodal_background.bed
1378 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~0.bed
8721 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~0_background.bed
7650 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1.bed
2449 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1_background.bed
2015 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_bimodal.bed
15932 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_bimodal_background.bed
2407 /projects/ps-yeo

In [4]:
mkdir /home/obotvinnik/projects/singlecell_pnms/analysis/homer/

mkdir: cannot create directory `/home/obotvinnik/projects/singlecell_pnms/analysis/homer/': File exists


## HOMER enrichment


In [5]:
homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir

### First extract upstream/downstream intron sequences

In [6]:
import os
import glob
import pybedtools

nt = 200

directions = 'upstream', 'downstream'

for bedfile in glob.iglob('{}/skipped_exon_exon2*.bed'.format(bed_folder)):
    if 'stream' in bedfile:
        continue
    
    basename = os.path.basename(bedfile)
    print basename
    prefix = basename.split('.bed')[0]
    bed = pybedtools.BedTool(bedfile)
    
    for direction in directions:
        # Get downstream intron
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, g=pybedtools.chromsizes('hg19'))
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, g=pybedtools.chromsizes('hg19'))
            
    # Saved every exon that was exactly upstream or downstream of a junction,
    # So when taking the flanking sequence, there's a lot of repetition
    unique = pybedtools.BedTool(list(set(x for x in intron)))
    
    
    print '\t', len(intron), len(unique)
    saveas = '{}/{}_{}{}.bed'.format(bed_folder, prefix, direction, nt)
    print '\t', saveas
    unique.saveas(saveas)
    ! wc -l $saveas

skipped_exon_exon2_NPC_bimodal_background.bed
	15932 15932
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_bimodal_background_downstream200.bed
15932 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_bimodal_background_downstream200.bed
skipped_exon_exon2_MN_~0_background.bed
	8721 8721
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~0_background_downstream200.bed
8721 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~0_background_downstream200.bed
skipped_exon_exon2_MN_~1.bed
	7650 7650
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1_downstream200.bed
7650 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1_downstream200.bed
skipped_exon_exon2_iPSC_bimodal.bed
	3220 3220
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped

## Enrichment of phenotype modality

#### Background: All other modalities, within that phenotype

In [7]:
import os
import fileinput

from gscripts.qtools import Submitter
import glob
import itertools

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6,7 -mset vertebrates -mis 1 -p {}'.format(n_processors)

commands = []


for bedfile in glob.iglob('{}/skipped_exon_exon2*.bed'.format(bed_folder)):

    basename = os.path.basename(bedfile)
    prefix = basename.split('.')[0]
    modality = prefix.split('_')[4]
    if 'background' in basename:
        continue
    
    background = bedfile.replace(modality, modality + '_background')
    
    out_dir = '{}/{}'.format(homer_dir, basename.replace('.bed', '_vs_alternative_background'))
    if os.path.exists('{}/homerResults.html'):
        print 'Already finished {}, moving on'.format(basename)
        continue
    command = '{} {} hg19 {} -bg {} {}'.format(
        findMotifsGenome, bedfile, out_dir, background, homer_flags)
    print '\n', command
    commands.append(command)
#     ! $command
jobname = 'homer_vs_alternative_background'
sub = Submitter(commands, jobname, walltime='2:00:00', queue='home-scrm',
               ppn=n_processors, write_and_submit=True, array=True,
               out_filename='{}/{}.out'.format(homer_dir, jobname),
               err_filename='{}/{}.err'.format(homer_dir, jobname),)

running 30 tasks as an array-job.
job ID: 3321465



/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_~0_downstream200.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/homer/skipped_exon_exon2_NPC_~0_downstream200_vs_alternative_background -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_NPC_~0_background_downstream200.bed -rna -len 4,5,6,7 -mset vertebrates -mis 1 -p 4

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/homer/skipped_exon_exon2_MN_~1_vs_alternative_background -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/skipped_exon_exon2_MN_~1_background.bed -rna -len 4,5,6,7 -mset vertebrates -mis 1 -p 4

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/